In [1]:
import pandas as pd

In [8]:
class sampop(object):
    def __init__ (self, df):
        '''A dataframe is essentially the input for the matrix'''
        self.df = pd.read_csv(df, index_col = 0)
        '''Call printsample to visualize .csv file in dataframe'''
        self.dfcopy = self.df.copy(deep = True)
        '''Use dataframe.copy to create a copy of original dataframe'''
        '''Or this step will mess up following calculations based on columns/rows'''
        self.dfcopy['Sum'] = self.dfcopy.sum(axis = 1)
        
    def printsample(self, stratum):
        '''stratum is a list users should enter. Elements in the list represent 
        observations in each stratum (pixels of each category).'''
        self.dfcopy['Stratum'] = stratum
        return self.dfcopy
    
    def population (self, stratum):
        '''stratum is a list users should enter. Elements in the list represent 
        observations in each stratum (pixels of each category).'''
        '''multiplier is a list of proportions between stratum and sample. 
        All values in 'sample' will multiply this ratio to get population values.'''
        multiplier = [i / j for i, j in zip(stratum, self.dfcopy['Sum'].to_list())] 
        '''create a new population dataframe that is derived from sample dataframe.'''
        population = self.dfcopy.iloc[:, 0:-1]
        population = population.mul(multiplier, axis = 'rows')
        population['Row_sum'] = population.sum(axis = 1)
        population.loc['Column_sum'] = population.sum(axis = 0)
        hits = []
        false_alarms = []
        populationsum = population['Row_sum'][:-1].to_list()
        for i in range(len(population.columns)-1):
            hits.append(population.iloc[i][i]) 
            false_alarms.append(populationsum[i] - hits[i])
        return population
        

In [9]:
sample = sampop('sample2.csv')
stratum = [30,60,25,30]
sample.printsample(stratum)

,j1,j2,j3,j4,Sum,Stratum
i1,1,0,2,0,3,30
i2,0,3,3,0,6,60
i3,2,0,3,0,5,25
i4,0,3,0,3,6,30


In [10]:
sample = sampop('sample2.csv')
stratum = [30,60,25,30]
sample.population(stratum)

,j1,j2,j3,j4,Row_sum
i1,10.0,0.0,20.0,0.0,30.0
i2,0.0,30.0,30.0,0.0,60.0
i3,10.0,0.0,15.0,0.0,25.0
i4,0.0,15.0,0.0,15.0,30.0
Column_sum,20.0,45.0,65.0,15.0,145.0
